In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_dataset = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = train_dataset.iloc[:, 1:2].values

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

In [2]:
train_X = []
train_y = []

for i in range(60,1258):
    train_X.append(training_set_scaled[i-60:i, 0])
    train_y.append(training_set_scaled[i, 0])
train_X, train_y = np.array(train_X), np.array(train_y)

train_X = np.reshape(train_X , (train_X.shape[0], train_X.shape[1],1))

In [3]:
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "c:\python37\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\python37\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\python37\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "c:\python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "c:\python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons and solutions.  Include the entire stack tra

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True , input_shape = (train_X.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50 , return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units =1))

regressor.compile(optimizer = 'adam' ,loss = 'mean_squared_error')

regressor.fit(train_X, train_y, epochs = 100, batch_size = 24)

In [ ]:
test_dataset = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = test_dataset.iloc[:, 1:2].values

dataset_total = pd.concat((train_dataset['Open'], test_dataset["Open"]),axis = 0)
inputs = dataset_total[len(dataset_total) - len(test_dataset) -60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

test_X = []
for i in range(60,80):
    test_X.append(inputs[i-60:i, 0])
test_X = np.array(test_X)
test_X = np.reshape(test_X , (test_X.shape[0], test_X.shape[1],1))
predicted_stock_price = regressor.predict(test_X)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price , color = 'red' ,label = 'Google Stock Price (Real)')
plt.plot(predicted_stock_price , color = 'green' , label = 'Google Stock Price (Predicted)')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()